<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=60a479843569f5c5f8ac278f242371304070b4461d920d9f6bb453d6dc501e8a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 11:18:50
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 1.22 L (0.85%)
Current PnL: -21.63 L (-14.18%)
CY Booked + Current PnL: -8.11 L (-5.32%)
-------------------
Total profit:  1.89 L
Total loss:  -23.52 L
-------------------
Total Booked + Current PnL: 18.87 L (15.02%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.32%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 88.30 L (60.9%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.35%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,101.71,51.0,M-SC,2.55,86912.0,-13865.0,13958.0,0.27,-13.76,16.06,0.09,245.0,-0.99,0.61,14.18,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.30,88170.0,-2508.0,17414.0,-0.54,-2.77,19.75,16.44,101.0,-0.14,0.61,16.85,X40,ATH,PHARMA
51,NESTLEIND,1377.00,-2.37,66.0,X-LC,8.60,300620.0,35194.0,21585.0,1.31,13.26,7.18,21.40,11.0,1.63,2.09,20.78,XY25,NTT,FMCG
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199047.0,-1091.0,21975.0,0.05,-0.55,11.04,10.44,4.0,-0.05,1.39,4.99,X40,NTT,FMCG
34,ICICIGI,2252.93,-13.38,57.0,X-MC,7.55,187469.0,13149.0,22046.0,0.99,7.54,11.76,20.19,91.0,0.60,1.31,23.80,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,ROUTE,2227.21,-56.81,40.0,H-SC,23.20,71210.0,-61856.0,153735.0,4.92,-46.49,215.89,69.05,118.0,-0.40,0.50,4.92,SR,ATH,IT
25,GILLETTE,11206.78,-16.29,33.0,X-MC,5.69,257400.0,3504.0,78816.0,3.57,1.38,30.62,32.42,105.0,0.04,1.79,16.69,X40,ATH,FMCG
16,CAMS,4762.00,-2.06,59.0,X-SC,1.23,262310.0,10454.0,51990.0,3.54,4.15,19.82,24.79,122.0,0.20,1.83,28.25,X40N,NTT,MISC
72,TATAELXSI,9161.00,-23.37,53.0,H-MC,6.32,103560.0,-28275.0,70504.0,3.16,-21.45,68.08,32.03,98.0,-0.40,0.72,16.00,OX40N,NTT,IT
39,INDUSINDBK,1800.00,294.26,76.0,L-MC,7.14,48404.0,-28802.0,54193.0,2.80,-37.31,111.96,32.89,258.0,-0.53,0.34,33.30,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,671.64,-18.08,48.0,X-LC,5.68,294820.0,-21022.0,133701.0,-1.81,-6.66,45.35,35.67,5.0,-0.16,2.05,6.28,X40N,ATH,FMCG
80,VALIANTORG,838.00,-275.92,35.0,H-SC,11.92,97852.0,-65753.0,187064.0,-0.96,-40.19,191.17,74.15,139.0,-0.35,0.68,25.43,XR,NTT,CHEMICALS
75,TMPV,1065.00,NaN,52.0,NaN,NaN,211097.0,-186547.0,344827.0,-0.78,-46.91,163.35,39.81,54.0,-0.54,1.47,3.47,XY24,NTT,AUTO
83,WHIRLPOOL,2270.00,-45.46,36.0,M-SC,8.83,113050.0,-1748.0,95787.0,-0.73,-1.52,84.73,81.92,223.0,-0.02,0.79,33.83,XR,NTT,DURABLES
70,SYMPHONY,1306.00,-44.94,42.0,M-SC,15.79,117310.0,-53831.0,53775.0,-0.70,-31.45,45.84,-0.03,196.0,-1.00,0.82,2.39,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.79,54.0,M-LC,5.74,151652.0,707.0,108750.0,1.20,0.47,71.71,72.51,53.0,0.01,1.06,6.52,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-8.62,40.0,H-MC,6.00,210690.0,1116.0,73552.0,1.54,0.53,34.91,35.63,NaN,0.02,1.47,17.29,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,101.71,51.0,M-SC,2.55,86912.0,-13865.0,13958.0,0.27,-13.76,16.06,0.09,245.0,-0.99,0.61,14.18,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,127.12,77.0,M-SC,4.51,148825.0,-25734.0,25762.0,-0.20,-14.74,17.31,0.02,221.0,-1.00,1.04,29.45,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,41.0,H-SC,2.30,219780.0,-49887.0,86220.0,0.14,-18.50,39.23,13.47,138.0,-0.58,1.53,11.75,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.37,53.0,H-MC,6.32,103560.0,-28275.0,70504.0,3.16,-21.45,68.08,32.03,98.0,-0.40,0.72,16.00,OX40N,NTT,IT
66,SIS,528.0,2029.02,48.0,H-SC,2.20,85922.0,-25110.0,48718.0,0.88,-22.62,56.70,21.26,156.0,-0.52,0.60,16.09,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,1286.17,-8.62,40.0,H-MC,6.00,210690.0,1116.0,73552.0,1.54,0.53,34.91,35.63,NaN,0.02,1.47,17.29,AR,ATH,PHARMA
84,WIPRO,420.00,-13.79,54.0,M-LC,5.74,151652.0,707.0,108750.0,1.20,0.47,71.71,72.51,53.0,0.01,1.06,6.52,XR,NTT,IT
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.91,132553.0,9217.0,122413.0,2.00,7.47,92.35,106.72,119.0,0.08,0.92,32.60,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,55.0,M-SC,35.64,160272.0,11052.0,133122.0,0.98,7.41,83.06,96.62,208.0,0.08,1.12,73.45,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.91,132553.0,9217.0,122413.0,2.00,7.47,92.35,106.72,119.0,0.08,0.92,32.60,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-8.62,40.0,H-MC,6.00,210690.0,1116.0,73552.0,1.54,0.53,34.91,35.63,NaN,0.02,1.47,17.29,AR,ATH,PHARMA
84,WIPRO,420.00,-13.79,54.0,M-LC,5.74,151652.0,707.0,108750.0,1.20,0.47,71.71,72.51,53.0,0.01,1.06,6.52,XR,NTT,IT
83,WHIRLPOOL,2270.00,-45.46,36.0,M-SC,8.83,113050.0,-1748.0,95787.0,-0.73,-1.52,84.73,81.92,223.0,-0.02,0.79,33.83,XR,NTT,DURABLES
47,KPIGREEN,731.05,2.88,44.0,H-SC,10.17,122749.0,-2548.0,61473.0,0.16,-2.03,50.08,47.03,141.0,-0.04,0.86,53.69,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86918.0,-41752.0,87048.0,0.03,-32.45,100.15,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-24.16,31.0,X-MC,4.71,190282.0,-10065.0,40568.0,-0.53,-5.02,21.32,15.23,178.0,-0.25,1.33,25.61,X40N,NTT,REALTY
1,ABB,7934.00,-42.78,32.0,H-LC,5.01,238968.0,-22651.0,141875.0,0.00,-8.66,59.37,45.57,7.0,-0.16,1.67,1.68,AR,NTT,ELECTRICAL
26,GLAXO,3466.20,-20.13,33.0,X-MC,4.82,202000.0,2716.0,75306.0,1.97,1.36,37.28,39.15,96.0,0.04,1.41,30.76,X40,ATH,PHARMA
23,DMART,5391.45,-20.17,33.0,X-LC,6.38,203505.0,-6902.0,66058.0,-0.05,-3.28,32.46,28.12,38.0,-0.10,1.42,19.57,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199047.0,-1091.0,21975.0,0.05,-0.55,11.04,10.44,4.0,-0.05,1.39,4.99,X40,NTT,FMCG
53,PGHH,17907.65,-32.66,34.0,X-MC,1.12,195090.0,-5730.0,73529.0,0.68,-2.85,37.69,33.76,80.0,-0.08,1.36,2.20,X40,ATH,FMCG
65,SIEMENS,4671.50,-5.61,39.0,H-LC,1.68,151585.0,-34510.0,81992.0,0.32,-18.54,54.09,25.51,15.0,-0.42,1.06,11.71,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-8.59,39.0,X-MC,3.30,304941.0,-23050.0,129569.0,0.59,-7.03,42.49,32.48,92.0,-0.18,2.13,3.59,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-7.09,60.0,X-LC,3.47,243500.0,9356.0,39277.0,0.08,4.00,16.13,20.77,86.0,0.24,1.70,11.19,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86918.0,-41752.0,87048.0,0.03,-32.45,100.15,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,34.0,X-SC,37.12,48322.0,-16684.0,167610.0,0.99,-25.67,346.86,232.18,198.0,-0.10,0.34,0.99,XY24,NTT,MISC
1,ABB,7934.00,-42.78,32.0,H-LC,5.01,238968.0,-22651.0,141875.0,0.00,-8.66,59.37,45.57,7.0,-0.16,1.67,1.68,AR,NTT,ELECTRICAL
10,BAJAJHFL,181.50,-15.96,39.0,X-MC,6.32,177202.0,-23304.0,120462.0,0.34,-11.62,67.98,48.45,90.0,-0.19,1.23,2.18,X40N,ATH,FINANCE
53,PGHH,17907.65,-32.66,34.0,X-MC,1.12,195090.0,-5730.0,73529.0,0.68,-2.85,37.69,33.76,80.0,-0.08,1.36,2.20,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,34.0,X-SC,37.12,48322.0,-16684.0,167610.0,0.99,-25.67,346.86,232.18,198.0,-0.10,0.34,0.99,XY24,NTT,MISC
58,RELAXO,1176.00,-40.17,45.0,X-SC,3.69,81592.0,-63568.0,139498.0,0.49,-43.79,170.97,52.31,136.0,-0.46,0.57,9.22,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.30,88170.0,-2508.0,17414.0,-0.54,-2.77,19.75,16.44,101.0,-0.14,0.61,16.85,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86918.0,-41752.0,87048.0,0.03,-32.45,100.15,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
33,HONAUT,58357.33,-24.11,38.0,X-SC,0.93,106950.0,-20988.0,68127.0,0.41,-16.41,63.70,36.84,143.0,-0.31,0.75,9.49,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,62.0,X-LC,8.96,292678.0,-53278.0,119969.0,2.19,-15.40,40.99,19.28,1.0,-0.44,2.04,8.20,X40,ATH,IT
40,INFY,2275.00,-15.29,67.0,X-LC,4.54,330171.0,17283.0,154388.0,1.29,5.52,46.76,54.87,3.0,0.11,2.30,14.26,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199047.0,-1091.0,21975.0,0.05,-0.55,11.04,10.44,4.0,-0.05,1.39,4.99,X40,NTT,FMCG
81,VBL,671.64,-18.08,48.0,X-LC,5.68,294820.0,-21022.0,133701.0,-1.81,-6.66,45.35,35.67,5.0,-0.16,2.05,6.28,X40N,ATH,FMCG
1,ABB,7934.00,-42.78,32.0,H-LC,5.01,238968.0,-22651.0,141875.0,0.00,-8.66,59.37,45.57,7.0,-0.16,1.67,1.68,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,41.0,L-SC,26.23,76301.0,-37248.0,77301.0,-0.40,-32.80,101.31,35.27,268.0,-0.48,0.53,86.75,XR,NTT,HOTELS
7,ASIANTILES,137.00,7583.33,68.0,L-SC,15.01,84281.0,-9529.0,86144.0,0.37,-10.16,102.21,81.67,269.0,-0.11,0.59,63.21,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,49.0,H-SC,7.88,91680.0,-6300.0,27907.0,1.04,-6.43,30.44,22.05,152.0,-0.23,0.64,33.26,XR,ATH,FINANCE
47,KPIGREEN,731.05,2.88,44.0,H-SC,10.17,122749.0,-2548.0,61473.0,0.16,-2.03,50.08,47.03,141.0,-0.04,0.86,53.69,MH,ATH,POWER
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.91,132553.0,9217.0,122413.0,2.00,7.47,92.35,106.72,119.0,0.08,0.92,32.60,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,521.00,70.25,59.0,H-SC,6.57,147430.0,-35345.0,146413.0,1.50,-19.34,99.31,60.77,125.0,-0.24,1.03,32.77,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7583.33,68.0,L-SC,15.01,84281.0,-9529.0,86144.0,0.37,-10.16,102.21,81.67,269.0,-0.11,0.59,63.21,XR,NTT,CERAMICS
38,INDIGOPNTS,1408.00,127.12,77.0,M-SC,4.51,148825.0,-25734.0,25762.0,-0.20,-14.74,17.31,0.02,221.0,-1.00,1.04,29.45,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,24.49,71.0,X-SC,6.90,220555.0,29549.0,28143.0,1.83,15.47,12.76,30.21,157.0,1.05,1.54,39.58,X40N,NTT,FINANCE
6,ASIANPAINT,3465.66,-7.05,75.0,X-LC,6.83,234608.0,-17160.0,70359.0,0.33,-6.82,29.99,21.13,27.0,-0.24,1.63,26.19,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.89
1,25,44.57
2,50,76.31


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.59
LC    27.90
MC    27.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.51
X40      20.74
X40N     12.42
XY25     10.20
XR        9.70
AR        9.17
OX40N     6.58
X200      1.83
SR        1.04
MH        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.94
X-MC    22.93
X-LC    19.90
M-SC    12.56
X-SC     5.64
H-LC     4.56
H-MC     3.82
M-LC     2.45
L-SC     1.45
L-LC     0.99
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.90,-3.78,37.34
IT,12.78,-14.65,76.02
FINANCE,12.15,-8.45,57.60
MISC,7.71,-17.41,76.63
PAINTS,5.75,-11.76,28.41
ELECTRICAL,5.61,-12.61,52.53
INSURANCE,4.47,1.43,33.30
PHARMA,3.49,0.26,33.20
AUTO,3.23,-50.06,112.18


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3218987.0,20
XR,1267623.0,13
AR,1256326.0,10
X40,993198.0,14
X40N,687824.0,9
OX40N,541561.0,9
XY25,469794.0,7
SR,272342.0,2
MH,61473.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3398185.0,24
X-MC,1396339.0,16
M-SC,1244920.0,15
X-LC,795854.0,11
X-SC,542416.0,6
H-MC,324434.0,3
H-LC,284873.0,3
L-SC,252860.0,3
M-LC,154376.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1172345.0      6
           AR         835615.0      5
           XR         798100.0      7
M-SC       XY24       722039.0      6
X-MC       XY24       564800.0      4
           X40        487355.0      8
X-LC       X40        418795.0      5
X-SC       X40N       287758.0      4
H-SC       SR         272342.0      2
           OX40N      258310.0      3
X-LC       X40N       239036.0      3
H-LC       AR         223867.0      2
X-MC       XY25       183154.0      2
H-MC       XY24       180378.0      1
X-SC       XY24       167610.0      1
L-SC       XR         163445.0      2
X-MC       X40N       161030.0      2
M-SC       XR         143135.0      2
           XY25       133122.0      1
           OX40N      123332.0      4
           AR         123292.0      2
M-LC       XR         108750.0      1
L-SC       OX40N       89415.0      1
X-SC       X40         87048.0      1
H-MC       AR          73552.0      1
X-LC       XY25        71035.0      2
H-MC       OX40N       70504.0      1
X-LC       XY24        66988.0      1
H-SC       MH          61473.0      1
H-LC       X200        61006.0      1
L-MC       XR          54193.0      1
M-LC       XY25        45626.0      1
L-LC       XY25        36857.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
